In [1]:
import torch
import pytorch_lightning as pl
import numpy as np
from gensim.models import Word2Vec
from train import Lightning

In [2]:
w2v = Word2Vec.load('data/w2v/news_200d.bin')
model = Lightning.load_from_checkpoint('lightning_logs/exp1_user_arch/version_0/checkpoints/epoch=0.ckpt')
doc_encoder = model.nrms.doc_encoder
w2id = {key: w2v.wv.vocab[key].index for key in w2v.wv.vocab}
id2w = {v: k for k, v in w2id.items()}


In [3]:
N = 10
result = []
projs = doc_encoder.get_ae_aspects()
for j, (proj, idx) in enumerate(zip(*projs)):
    idx = idx[-N:].detach().cpu().numpy()
    words = ', '.join([id2w[i] for i in idx])
    result.append(words)
result

['3q, wynwood, usl, suggestions, kiermaier, click, leaps, anti-government, colony, buries',
 'entrepreneurs, fulfillment, 115-99, vettel, fisheries, zimmer, politi, charlestown, 8-game, parades',
 "3,400, wynwood, 3q, marking, fleming, badin, 'ready, parades, versatility, allegan",
 "'better, agusta, markus, tyrell, pahokee, shep, mv, gyms, gale, sielski",
 'click, gyms, accessory, holdings, kiermaier, binghamton, gale, uaw-gm, alpinestars, x-rays',
 '115-99, click, allegan, mv, gyms, nicholls, binghamton, 65-yard, heinz, salads',
 'sielski, mv, marking, gyms, anti-government, barbershop, severino, zimmer, politi, allegan',
 "doritos, 'fox, owen, lorain, usl, 112-106, no-call, buries, parades, binghamton",
 'tfc, u-17, anti-government, 115-99, 112-106, aquidneck, ellum, pt, holdings, fleming',
 "ellum, badin, workshop, 'ready, pt, gale, borough-based, kiermaier, allegan, versatility",
 'allegan, xhaka, seizes, parades, anti-government, 65-yard, mane, broadcasters, 3q, fleming',
 'holdi

In [4]:
doc_encoder = model.nrms.doc_encoder

In [5]:
title = 'Best PS5 games : top PlayStation 5 titles to look forward to'.split()
idx = torch.tensor([w2id.get(w.lower(), 0) for w in title])
idx

tensor([   31, 10810,   241,     1,    32,  8881,    67,  4443,    12,   148,
         1089,    12])

In [6]:
o, score, _ = doc_encoder(idx.unsqueeze(0), False)
score = score.squeeze().detach().tolist()

In [7]:
list(zip(title, score))

[('Best', 0.1231062188744545),
 ('PS5', 0.1617722511291504),
 ('games', 0.027761921286582947),
 (':', 0.015886632725596428),
 ('top', 0.12275715917348862),
 ('PlayStation', 0.1066460907459259),
 ('5', 0.13037270307540894),
 ('titles', 0.0676967054605484),
 ('to', 0.010461097583174706),
 ('look', 0.04341239482164383),
 ('forward', 0.17113593220710754),
 ('to', 0.018990863114595413)]

In [8]:
title = 'NFL Week 9 Odds: Point Spreads, Betting Lines For All 14 Football Games'.split()
idx = torch.tensor([w2id.get(w.lower(), 0) for w in title])
o, score, _ = doc_encoder(idx.unsqueeze(0), False)
score = score.squeeze().detach().tolist()
list(zip(title, score))

[('Redskins', 0.08986718952655792),
 ('place', 0.09398113936185837),
 ('TE', 0.09424681216478348),
 ('Jordan', 0.11703678965568542),
 ('Reed', 0.08849827945232391),
 ('on', 0.08922896534204483),
 ('injured', 0.053749967366456985),
 ('reserve,', 0.0956740751862526),
 ('ending', 0.09256748110055923),
 ('his', 0.1126408576965332),
 ('season', 0.07250846177339554)]